# IMPORTS

In [196]:
%reload_ext autoreload
%autoreload 2
# GENERAL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

# LOADING DATA
import requests
import os
import shutil

# GEOSPATIAL DATA
from shapely.geometry import Point, LineString, MultiPoint

# PLOTTING DATA
from folium import Map, CircleMarker, Vega, Popup, Marker, PolyLine, Icon, Choropleth, LayerControl
from folium.plugins import MarkerCluster, HeatMap, BeautifyIcon
import shapely
import matplotlib
from ipywidgets import interact
import seaborn as sns

# STATS
import math
import stats

# ML
import scipy
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler



# Preprocessing 

## database n.1 : ROAD RISK ~ Shape file

The first database we are cleaning is a shape file containing segments of road in Porto and Lisbon. 
In the next steps we are preparing the dataset for the future analysis.

### Extracting Data

In [197]:
# link = 'https://wdl-data.fra1.digitaloceanspaces.com/pse/m_risk_prfile.zip'
# s = requests.get(link).content

In [198]:
def first_df(path):
    geo = gpd.read_file(path)
    df = pd.DataFrame(geo).drop(columns='Link_ID')
    df_ren = df.rename(columns={
                        'Daily_Aver':'Daily_Average_Traffic_Intensity',
                        'Average_Ve':'Average_Velocity_of_Vehicle_Traffic',
                        'Median_of_':'Median_of_velocity_of_Vehicle_Traffic',
                        'First_Quar': 'FirstQuartil_of_velocity_of_Vehicle_Traffic',
                        'Third_Quar': 'ThirdQuartil_of_velocity_of_Vehicle_Traffic'
                    })
    return df_ren
    
df = first_df('wdl_data/m_risk_prfile.geojson') 

- Now we have a dataset containing the same columns but renamed. 
- We also drop the 'Link_ID' column as contain the unique id of the streat, information already present in linkid column

To have a better understanding on our data and to avoid errors during our analysis we need to investigate it with general statistics

### Remove outliers: 

In [199]:
df.describe()

,linkid,Daily_Average_Traffic_Intensity,Average_Velocity_of_Vehicle_Traffic,Median_of_velocity_of_Vehicle_Traffic,FirstQuartil_of_velocity_of_Vehicle_Traffic,ThirdQuartil_of_velocity_of_Vehicle_Traffic,Func_Class,Speed_Cat
count,3.467800e+04,34678.000000,34678.000000,34678.000000,34678.000000,34678.000000,34678.000000,34678.000000
mean,8.958206e+08,3340.417942,56.816834,56.463409,43.822041,68.091844,2.684613,4.904781
std,2.355910e+08,2725.873982,51.983670,26.240876,24.442204,30.985191,0.538658,1.520568
min,8.021682e+07,14.435864,-401.703724,1.000000,-392.500000,1.000000,1.000000,2.000000
25%,7.364832e+08,1903.398108,38.315321,38.250000,26.000000,48.000000,2.000000,4.000000
50%,9.067377e+08,2644.529317,49.966126,50.000000,38.875000,60.333333,3.000000,6.000000
75%,1.154997e+09,3897.886608,69.511585,71.000000,56.000000,85.000000,3.000000,6.000000
max,1.223731e+09,49309.806935,6357.022296,1326.250000,143.000000,2605.000000,3.000000,7.000000


- Regarding the columns we know that they report values in km/h: many of the min and max we can observe thank to describe function don't make sense. 
- We need to operate on them as they are **outliers**

In [200]:
def rm_out(df):
    for i in df.columns.drop(['linkid', 'Daily_Average_Traffic_Intensity','geometry']):
        lb = 0
        ub = 180
#         print(lb, ub)
        df[i] = df[i].mask(df[i] < lb) 
        df[i] = df[i].mask(df[i] > ub) 
    return df

data = rm_out(df)

**If we run describe again we will see that the data regarding velocity has just feasible values.**

In [201]:
data.describe()

,linkid,Daily_Average_Traffic_Intensity,Average_Velocity_of_Vehicle_Traffic,Median_of_velocity_of_Vehicle_Traffic,FirstQuartil_of_velocity_of_Vehicle_Traffic,ThirdQuartil_of_velocity_of_Vehicle_Traffic,Func_Class,Speed_Cat
count,3.467800e+04,34678.000000,34633.000000,34675.000000,34677.000000,34674.000000,34678.000000,34678.000000
mean,8.958206e+08,3340.417942,56.112805,56.402803,43.834624,67.959767,2.684613,4.904781
std,2.355910e+08,2725.873982,24.346245,25.054113,24.329987,26.706718,0.538658,1.520568
min,8.021682e+07,14.435864,1.000000,1.000000,0.000000,1.000000,1.000000,2.000000
25%,7.364832e+08,1903.398108,38.317003,38.250000,26.000000,48.000000,2.000000,4.000000
50%,9.067377e+08,2644.529317,49.961538,50.000000,38.875000,60.333333,3.000000,6.000000
75%,1.154997e+09,3897.886608,69.447459,71.000000,56.000000,85.000000,3.000000,6.000000
max,1.223731e+09,49309.806935,179.691892,143.250000,143.000000,164.000000,3.000000,7.000000


### Handling duplicates:

In [202]:
len(data) == len(data.drop_duplicates())

True

There are ***no duplicates*** in our dataset

### Handling missing values:

In [203]:
data.isnull().sum().sort_values(ascending=False) , f'Total of data points : {data.shape[0]}'

(Average_Velocity_of_Vehicle_Traffic            45
 ThirdQuartil_of_velocity_of_Vehicle_Traffic     4
 Median_of_velocity_of_Vehicle_Traffic           3
 FirstQuartil_of_velocity_of_Vehicle_Traffic     1
 linkid                                          0
 Daily_Average_Traffic_Intensity                 0
 Func_Class                                      0
 Speed_Cat                                       0
 geometry                                        0
 dtype: int64,
 'Total of data points : 34678')

As we can see the total highest number of missing values detected for column is 45 on a total number of rows of 34'678.
- The missing value for Average Velocity e the ones in Speed Difference Mean are the same (one column is created from the other one)
- The missing values of other column can be easily deleted

**As we are handling data regarding AVERAGE velocity we can easily substitute the missing values with the mean of the corresponding column**

In [204]:
def handling_missing(data):
    imputer = SimpleImputer()
    data['Average_Velocity_of_Vehicle_Traffic']=imputer.fit_transform(data[['Average_Velocity_of_Vehicle_Traffic']])
    return data.dropna()
data = handling_missing(data)

In [205]:
data.isnull().sum()

linkid                                         0
Daily_Average_Traffic_Intensity                0
Average_Velocity_of_Vehicle_Traffic            0
Median_of_velocity_of_Vehicle_Traffic          0
FirstQuartil_of_velocity_of_Vehicle_Traffic    0
ThirdQuartil_of_velocity_of_Vehicle_Traffic    0
Func_Class                                     0
Speed_Cat                                      0
geometry                                       0
dtype: int64

### Feature creation:

Now looking at our data we need to search for a target that in the next step we will use in our model.
Most common causes of Accidents:
- Over Speeding.
- Drunken Driving.
- Distractions to Driver.
- Red Light Jumping.
- Avoiding Safety Gears like Seat belts and Helmets.
- Non-adherence to lane driving and overtaking in a wrong manner.

The first cause is always the **over-speed** that can be connected with one of the above causes. 
For this reason we decide to investigate and use as target information regarding the velocity.

- Speed_Cat (described in the excel above)
- Average Velocity of Vehicle Traffic 
- Median of velocity of Vehicle Traffic

We will create a dictionary that, from the information contained in the excel can describe the type of street regarding the max velocity allowed in there. 


In [206]:
speed_explanation = pd.read_excel('wdl_dict/Dictionary_Risk_Profiles.xlsx', sheet_name='SpeedCat')
speed_explanation

,Speed Cat,Speed range in km/h
0,1,>130 km/h
1,2,101-130 km/h
2,3,91-100 km/h
3,4,71-90 km/h
4,5,51-70 km/h
5,6,31-50 km/h
6,7,11-30 km/h
7,8,<11 km/h


 from the table above we can create a dictionary.
1. count values for category
2. translate the speed range in actual number

In [207]:
data.Speed_Cat.value_counts()

6    20307
4     5273
2     5123
3     1843
5     1770
7      357
Name: Speed_Cat, dtype: int64

**NO need of mapping for label 1 and 8**

In [208]:
max_speed_dict = {2:130,3:100,4:90,5:70,6:50,7:30}

In [209]:
def target_creation(data):
    data['Max_speed'] = data['Speed_Cat'].map(max_speed_dict)
    data['Speed_Diff_Mean'] = data['Max_speed'] - data['Average_Velocity_of_Vehicle_Traffic']
    data['Speed_Diff_Median'] = data['Max_speed'] - data['Median_of_velocity_of_Vehicle_Traffic']
    return data
data = target_creation(data)

<ipython-input-209-63521103fb59>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Max_speed'] = data['Speed_Cat'].map(max_speed_dict)
<ipython-input-209-63521103fb59>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Speed_Diff_Mean'] = data['Max_speed'] - data['Average_Velocity_of_Vehicle_Traffic']
<ipython-input-209-63521103fb59>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [210]:
data[['Speed_Cat','Max_speed', 'Speed_Diff_Mean','Speed_Diff_Median']].head(10)

,Speed_Cat,Max_speed,Speed_Diff_Mean,Speed_Diff_Median
0,6,50,4.791284,5.535714
1,4,90,10.436692,7.250000
2,6,50,-15.955069,-17.333333
3,6,50,2.466089,6.000000
4,6,50,-1.375291,-3.250000
5,6,50,17.667821,18.333333
6,6,50,19.613591,22.937500
7,6,50,26.714286,39.000000
8,6,50,6.476409,4.000000
9,6,50,-11.897045,1.346154


**our first target will be the difference between the mean of velocity and the max speed**

### Scaling features:

We are now ready to scale our dataframe to have a distribution *around* the mean.

1. We need to separate numerical and categorical column
2. We are going to use the Min-Max Scaling method for the numerical ones: is the one that is commonly used distance based algorithms, as k-means that is one of the possible analysis we are taking in consideration.  
3. For the categorical ones we'll use the OneHotEncoding method (for each label in each category creates a different column)

We could also operate this step all together but is important for us to know which column belong to each of the different classes inside the categorical feature. 
**To do so we need to operate for each categorical separately**

In [211]:
def scaling_numerical(data):
    numerical = data.columns.drop(['geometry','linkid','Speed_Cat', 'Func_Class'])
    scaler = StandardScaler()
    data_scaled = data.copy()
    for column in numerical:
        scaler.fit(data_scaled[[column]])
        data_scaled[column]=scaler.transform(data_scaled[[column]]) 
    return data_scaled
data_scaled = scaling_numerical(data)

In [212]:
data.drop(columns=['geometry','linkid','Speed_Cat', 'Func_Class']).head(1)

,Daily_Average_Traffic_Intensity,Average_Velocity_of_Vehicle_Traffic,Median_of_velocity_of_Vehicle_Traffic,FirstQuartil_of_velocity_of_Vehicle_Traffic,ThirdQuartil_of_velocity_of_Vehicle_Traffic,Max_speed,Speed_Diff_Mean,Speed_Diff_Median
0,6224.778569,45.208716,44.464286,28.0,59.0,50,4.791284,5.535714


In [213]:
data_scaled.drop(columns=['geometry','linkid','Speed_Cat', 'Func_Class']).head(1)

,Daily_Average_Traffic_Intensity,Average_Velocity_of_Vehicle_Traffic,Median_of_velocity_of_Vehicle_Traffic,FirstQuartil_of_velocity_of_Vehicle_Traffic,ThirdQuartil_of_velocity_of_Vehicle_Traffic,Max_speed,Speed_Diff_Mean,Speed_Diff_Median
0,1.058039,-0.448154,-0.476554,-0.650809,-0.33552,-0.716756,-0.549408,-0.506457


**Working with the categorical features the first thing we need to do is to understand the distribuition within the labels**

In [214]:
data.Func_Class.value_counts() , data.Speed_Cat.value_counts()

(3    25023
 2     8363
 1     1287
 Name: Func_Class, dtype: int64,
 6    20307
 4     5273
 2     5123
 3     1843
 5     1770
 7      357
 Name: Speed_Cat, dtype: int64)

- Functional Class has just 3 possible label for the street that we can understand better looking at the excel

In [215]:
func_explanation = pd.read_excel('wdl_dict/Dictionary_Risk_Profiles.xlsx', sheet_name='Func_Class')
for i,el in enumerate(func_explanation['Description']):
    print(f'Class n.{i+1} : {el} \n')

Class n.1 : These roads are meant for high volume, maximum speed traffic between and through major metropolitan areas. There are very few, if any, speed changes. Access to this road is usually controlled. 

Class n.2 : These roads are used to channel traffic to Main Roads (FRC1) for travel between and through cities in the shortest amount of time. There are very few, if any speed changes. 

Class n.3 : These roads interconnect First Class Roads (FRC2) and provide a high volume of traffic movement at a lower level of mobility than First Class Roads (FRC2). 



With this new and deeper understanding of the distribution and the meaning of the category (*NB: regarding speed_cat we can look back at the point **1.1.5 "Feature creation"** to get these informations)* we can now progress with our transformations.  

In [216]:
def scaling_categorical(data):
    ohe = OneHotEncoder(sparse = False)
    ohe.fit(data[['Func_Class']])
    func_encoded = ohe.transform(data[['Func_Class']])
    data["func_1"],data["func_2"],data['func_3'] = func_encoded.T
    ohe = OneHotEncoder(sparse = False)
    ohe.fit(data[['Speed_Cat']])
    speed_encoded = ohe.transform(data[['Speed_Cat']])
    data["speed_2"],data["speed_3"],data["speed_4"],\
    data["speed_5"], data["speed_6"], data["speed_7"]= speed_encoded.T
    return data 

In [217]:
data_scaled = scaling_categorical(data_scaled)

### Preprocessed Dataframe: 

In [218]:
data_scaled.columns

Index(['linkid', 'Daily_Average_Traffic_Intensity',
       'Average_Velocity_of_Vehicle_Traffic',
       'Median_of_velocity_of_Vehicle_Traffic',
       'FirstQuartil_of_velocity_of_Vehicle_Traffic',
       'ThirdQuartil_of_velocity_of_Vehicle_Traffic', 'Func_Class',
       'Speed_Cat', 'geometry', 'Max_speed', 'Speed_Diff_Mean',
       'Speed_Diff_Median', 'func_1', 'func_2', 'func_3', 'speed_2', 'speed_3',
       'speed_4', 'speed_5', 'speed_6', 'speed_7'],
      dtype='object')

## Feature engineering

We assume that ***over speeding*** is the main reason for road hazards. Over speeding behavior can be extracted by the deltas between the road's speed category and its actual average speed observations as processed in column ```Speed_Diff_Mean```. <br>

Over speeding can be harnessed among others by the roads environment [Source](https://www.tandfonline.com/doi/abs/10.1080/014416499295420). People chose their speeding behavior not only by speed limits but also by their assessment of the road's quality and the surrounding environment.<br>

Therefore we chose to gather more information about POIs, amenities and public buildings in the surrounding of the provided road segments. Those can be acquired through OSM sources.

### Scaling data set to Lisbon

#########################################################<br>
***_----------FILL WITH EXPLANATION SISTO------------_***<br>
#########################################################

In [219]:
# Transforming pandas df to geopandas df
geo_df = gpd.GeoDataFrame(data_scaled)
geo_df.geometry[0].type

'MultiLineString'

In [220]:
## Filtering only lisbon data inside the circle of 38.72526068747401, -9.142352617846093 with buffer '1'
circle_lisbon = Point(-9.142352617846093, 38.72526068747401).buffer(1)
geo_lis = geo_df[geo_df.geometry.within(circle_lisbon)]
# no immediate usage of this pd.DataFrame
df_lis = pd.DataFrame(geo_lis).drop(columns=['geometry', 'linkid'])

In [221]:
print(f'The new data set has {df_lis.shape[0]} rows as opposed to the original set with {geo_df.shape[0]} rows')

The new data set has 17688 rows as opposed to the original set with 34673 rows


### Loading POIs from pre-processed OSM file

In [222]:
# due to a large data set the file has to be loaded as pd.DataFrame before it can be transformed into a gpd.GeoDataFrame
poi_df = pd.read_csv('map_data/gdf_lis_poly.csv')
poi_df['geometry'] = poi_df['geometry'].apply(shapely.wkt.loads)
poi_gdf = gpd.GeoDataFrame(poi_df, crs='OGC:CRS84')

/Users/moritzgeiger/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3,5,6,7,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [223]:
print(f'The data set of POIs in the Lisbon region has {poi_gdf.shape[0]} individual points which can be merged with our data set.' )

The data set of POIs in the Lisbon region has 719511 individual points which can be merged with our data set.


In [224]:
poi_gdf.geometry.type.value_counts()

Polygon               420865
LineString            195623
Point                  99120
GeometryCollection      3261
MultiLineString          642
dtype: int64

***Note***<br>
For now we will only be focussing on the geometrical points in the OSM data, not on polygons or line strings.

In [225]:
# filtering down to shapely.geometry.Points
gdf_points = poi_gdf[poi_gdf['geometry'].type == 'Point'].reset_index()
gdf_points.columns

Index(['index', 'geometry', 'osm_id', 'name', 'amenity', 'barrier', 'building',
       'highway', 'landuse', 'man_made', 'natural', 'office'],
      dtype='object')

**Note**<br>
The points provided are categorized and stored in many columns. We will shrink this information to one column and fill it with all the important information about the point. <br>
Some points do not provide any information. Those ones will be dropped. 

In [226]:
# reducing geo_df columns, only leaving one valid column
def new_desc(geo):
    geo['desc_points'] = None
    # columns to be taken into consideration
    lst_cols = [  'amenity', 
                  'barrier', 
                  'building', 
                  'highway', 
                  'landuse', 
                  'man_made', 
                  'natural', 
                  'office']
    for c, row in geo.iterrows():
        concat_name = [f'feat_{i}_{row[i]}' for i in lst_cols if row[i] == row[i]]
        if len(concat_name) > 0:
            geo.at[c, 'desc_points'] = concat_name[0]
        else: 
            geo.at[c, 'desc_points'] = None
        print(f'done: {c}')
        
    
    geo = geo[['geometry', 'desc_points']]
    # drop empty descriptions
    geo = geo.dropna(subset=['desc_points'])
    geo= geo.reset_index(drop=True)
    
    return geo

In [227]:
# applying cleaning function to geo df
gdf_points_clean = new_desc(gdf_points)

done: 0
done: 1
done: 2
done: 3
done: 4
done: 5
done: 6
done: 7
done: 8
done: 9
done: 10
done: 11
done: 12
done: 13
done: 14
done: 15
done: 16
done: 17
done: 18
done: 19
done: 20
done: 21
done: 22
done: 23
done: 24
done: 25
done: 26
done: 27
done: 28
done: 29
done: 30
done: 31
done: 32
done: 33
done: 34
done: 35
done: 36
done: 37
done: 38
done: 39
done: 40
done: 41
done: 42
done: 43
done: 44
done: 45
done: 46
done: 47
done: 48
done: 49
done: 50
done: 51
done: 52
done: 53
done: 54
done: 55
done: 56
done: 57
done: 58
done: 59
done: 60
done: 61
done: 62
done: 63
done: 64
done: 65
done: 66
done: 67
done: 68
done: 69
done: 70
done: 71
done: 72
done: 73
done: 74
done: 75
done: 76
done: 77
done: 78
done: 79
done: 80
done: 81
done: 82
done: 83
done: 84
done: 85
done: 86
done: 87
done: 88
done: 89
done: 90
done: 91
done: 92
done: 93
done: 94
done: 95
done: 96
done: 97
done: 98
done: 99
done: 100
done: 101
done: 102
done: 103
done: 104
done: 105
done: 106
done: 107
done: 108
done: 109
done: 110


done: 1312
done: 1313
done: 1314
done: 1315
done: 1316
done: 1317
done: 1318
done: 1319
done: 1320
done: 1321
done: 1322
done: 1323
done: 1324
done: 1325
done: 1326
done: 1327
done: 1328
done: 1329
done: 1330
done: 1331
done: 1332
done: 1333
done: 1334
done: 1335
done: 1336
done: 1337
done: 1338
done: 1339
done: 1340
done: 1341
done: 1342
done: 1343
done: 1344
done: 1345
done: 1346
done: 1347
done: 1348
done: 1349
done: 1350
done: 1351
done: 1352
done: 1353
done: 1354
done: 1355
done: 1356
done: 1357
done: 1358
done: 1359
done: 1360
done: 1361
done: 1362
done: 1363
done: 1364
done: 1365
done: 1366
done: 1367
done: 1368
done: 1369
done: 1370
done: 1371
done: 1372
done: 1373
done: 1374
done: 1375
done: 1376
done: 1377
done: 1378
done: 1379
done: 1380
done: 1381
done: 1382
done: 1383
done: 1384
done: 1385
done: 1386
done: 1387
done: 1388
done: 1389
done: 1390
done: 1391
done: 1392
done: 1393
done: 1394
done: 1395
done: 1396
done: 1397
done: 1398
done: 1399
done: 1400
done: 1401
done: 1402

done: 2469
done: 2470
done: 2471
done: 2472
done: 2473
done: 2474
done: 2475
done: 2476
done: 2477
done: 2478
done: 2479
done: 2480
done: 2481
done: 2482
done: 2483
done: 2484
done: 2485
done: 2486
done: 2487
done: 2488
done: 2489
done: 2490
done: 2491
done: 2492
done: 2493
done: 2494
done: 2495
done: 2496
done: 2497
done: 2498
done: 2499
done: 2500
done: 2501
done: 2502
done: 2503
done: 2504
done: 2505
done: 2506
done: 2507
done: 2508
done: 2509
done: 2510
done: 2511
done: 2512
done: 2513
done: 2514
done: 2515
done: 2516
done: 2517
done: 2518
done: 2519
done: 2520
done: 2521
done: 2522
done: 2523
done: 2524
done: 2525
done: 2526
done: 2527
done: 2528
done: 2529
done: 2530
done: 2531
done: 2532
done: 2533
done: 2534
done: 2535
done: 2536
done: 2537
done: 2538
done: 2539
done: 2540
done: 2541
done: 2542
done: 2543
done: 2544
done: 2545
done: 2546
done: 2547
done: 2548
done: 2549
done: 2550
done: 2551
done: 2552
done: 2553
done: 2554
done: 2555
done: 2556
done: 2557
done: 2558
done: 2559

done: 3782
done: 3783
done: 3784
done: 3785
done: 3786
done: 3787
done: 3788
done: 3789
done: 3790
done: 3791
done: 3792
done: 3793
done: 3794
done: 3795
done: 3796
done: 3797
done: 3798
done: 3799
done: 3800
done: 3801
done: 3802
done: 3803
done: 3804
done: 3805
done: 3806
done: 3807
done: 3808
done: 3809
done: 3810
done: 3811
done: 3812
done: 3813
done: 3814
done: 3815
done: 3816
done: 3817
done: 3818
done: 3819
done: 3820
done: 3821
done: 3822
done: 3823
done: 3824
done: 3825
done: 3826
done: 3827
done: 3828
done: 3829
done: 3830
done: 3831
done: 3832
done: 3833
done: 3834
done: 3835
done: 3836
done: 3837
done: 3838
done: 3839
done: 3840
done: 3841
done: 3842
done: 3843
done: 3844
done: 3845
done: 3846
done: 3847
done: 3848
done: 3849
done: 3850
done: 3851
done: 3852
done: 3853
done: 3854
done: 3855
done: 3856
done: 3857
done: 3858
done: 3859
done: 3860
done: 3861
done: 3862
done: 3863
done: 3864
done: 3865
done: 3866
done: 3867
done: 3868
done: 3869
done: 3870
done: 3871
done: 3872

done: 4810
done: 4811
done: 4812
done: 4813
done: 4814
done: 4815
done: 4816
done: 4817
done: 4818
done: 4819
done: 4820
done: 4821
done: 4822
done: 4823
done: 4824
done: 4825
done: 4826
done: 4827
done: 4828
done: 4829
done: 4830
done: 4831
done: 4832
done: 4833
done: 4834
done: 4835
done: 4836
done: 4837
done: 4838
done: 4839
done: 4840
done: 4841
done: 4842
done: 4843
done: 4844
done: 4845
done: 4846
done: 4847
done: 4848
done: 4849
done: 4850
done: 4851
done: 4852
done: 4853
done: 4854
done: 4855
done: 4856
done: 4857
done: 4858
done: 4859
done: 4860
done: 4861
done: 4862
done: 4863
done: 4864
done: 4865
done: 4866
done: 4867
done: 4868
done: 4869
done: 4870
done: 4871
done: 4872
done: 4873
done: 4874
done: 4875
done: 4876
done: 4877
done: 4878
done: 4879
done: 4880
done: 4881
done: 4882
done: 4883
done: 4884
done: 4885
done: 4886
done: 4887
done: 4888
done: 4889
done: 4890
done: 4891
done: 4892
done: 4893
done: 4894
done: 4895
done: 4896
done: 4897
done: 4898
done: 4899
done: 4900

...

done: 93530
done: 93531
done: 93532
done: 93533
done: 93534
done: 93535
done: 93536
done: 93537
done: 93538
done: 93539
done: 93540
done: 93541
done: 93542
done: 93543
done: 93544
done: 93545
done: 93546
done: 93547
done: 93548
done: 93549
done: 93550
done: 93551
done: 93552
done: 93553
done: 93554
done: 93555
done: 93556
done: 93557
done: 93558
done: 93559
done: 93560
done: 93561
done: 93562
done: 93563
done: 93564
done: 93565
done: 93566
done: 93567
done: 93568
done: 93569
done: 93570
done: 93571
done: 93572
done: 93573
done: 93574
done: 93575
done: 93576
done: 93577
done: 93578
done: 93579
done: 93580
done: 93581
done: 93582
done: 93583
done: 93584
done: 93585
done: 93586
done: 93587
done: 93588
done: 93589
done: 93590
done: 93591
done: 93592
done: 93593
done: 93594
done: 93595
done: 93596
done: 93597
done: 93598
done: 93599
done: 93600
done: 93601
done: 93602
done: 93603
done: 93604
done: 93605
done: 93606
done: 93607
done: 93608
done: 93609
done: 93610
done: 93611
done: 93612
done

done: 94756
done: 94757
done: 94758
done: 94759
done: 94760
done: 94761
done: 94762
done: 94763
done: 94764
done: 94765
done: 94766
done: 94767
done: 94768
done: 94769
done: 94770
done: 94771
done: 94772
done: 94773
done: 94774
done: 94775
done: 94776
done: 94777
done: 94778
done: 94779
done: 94780
done: 94781
done: 94782
done: 94783
done: 94784
done: 94785
done: 94786
done: 94787
done: 94788
done: 94789
done: 94790
done: 94791
done: 94792
done: 94793
done: 94794
done: 94795
done: 94796
done: 94797
done: 94798
done: 94799
done: 94800
done: 94801
done: 94802
done: 94803
done: 94804
done: 94805
done: 94806
done: 94807
done: 94808
done: 94809
done: 94810
done: 94811
done: 94812
done: 94813
done: 94814
done: 94815
done: 94816
done: 94817
done: 94818
done: 94819
done: 94820
done: 94821
done: 94822
done: 94823
done: 94824
done: 94825
done: 94826
done: 94827
done: 94828
done: 94829
done: 94830
done: 94831
done: 94832
done: 94833
done: 94834
done: 94835
done: 94836
done: 94837
done: 94838
done

done: 95844
done: 95845
done: 95846
done: 95847
done: 95848
done: 95849
done: 95850
done: 95851
done: 95852
done: 95853
done: 95854
done: 95855
done: 95856
done: 95857
done: 95858
done: 95859
done: 95860
done: 95861
done: 95862
done: 95863
done: 95864
done: 95865
done: 95866
done: 95867
done: 95868
done: 95869
done: 95870
done: 95871
done: 95872
done: 95873
done: 95874
done: 95875
done: 95876
done: 95877
done: 95878
done: 95879
done: 95880
done: 95881
done: 95882
done: 95883
done: 95884
done: 95885
done: 95886
done: 95887
done: 95888
done: 95889
done: 95890
done: 95891
done: 95892
done: 95893
done: 95894
done: 95895
done: 95896
done: 95897
done: 95898
done: 95899
done: 95900
done: 95901
done: 95902
done: 95903
done: 95904
done: 95905
done: 95906
done: 95907
done: 95908
done: 95909
done: 95910
done: 95911
done: 95912
done: 95913
done: 95914
done: 95915
done: 95916
done: 95917
done: 95918
done: 95919
done: 95920
done: 95921
done: 95922
done: 95923
done: 95924
done: 95925
done: 95926
done

done: 97083
done: 97084
done: 97085
done: 97086
done: 97087
done: 97088
done: 97089
done: 97090
done: 97091
done: 97092
done: 97093
done: 97094
done: 97095
done: 97096
done: 97097
done: 97098
done: 97099
done: 97100
done: 97101
done: 97102
done: 97103
done: 97104
done: 97105
done: 97106
done: 97107
done: 97108
done: 97109
done: 97110
done: 97111
done: 97112
done: 97113
done: 97114
done: 97115
done: 97116
done: 97117
done: 97118
done: 97119
done: 97120
done: 97121
done: 97122
done: 97123
done: 97124
done: 97125
done: 97126
done: 97127
done: 97128
done: 97129
done: 97130
done: 97131
done: 97132
done: 97133
done: 97134
done: 97135
done: 97136
done: 97137
done: 97138
done: 97139
done: 97140
done: 97141
done: 97142
done: 97143
done: 97144
done: 97145
done: 97146
done: 97147
done: 97148
done: 97149
done: 97150
done: 97151
done: 97152
done: 97153
done: 97154
done: 97155
done: 97156
done: 97157
done: 97158
done: 97159
done: 97160
done: 97161
done: 97162
done: 97163
done: 97164
done: 97165
done

done: 98272
done: 98273
done: 98274
done: 98275
done: 98276
done: 98277
done: 98278
done: 98279
done: 98280
done: 98281
done: 98282
done: 98283
done: 98284
done: 98285
done: 98286
done: 98287
done: 98288
done: 98289
done: 98290
done: 98291
done: 98292
done: 98293
done: 98294
done: 98295
done: 98296
done: 98297
done: 98298
done: 98299
done: 98300
done: 98301
done: 98302
done: 98303
done: 98304
done: 98305
done: 98306
done: 98307
done: 98308
done: 98309
done: 98310
done: 98311
done: 98312
done: 98313
done: 98314
done: 98315
done: 98316
done: 98317
done: 98318
done: 98319
done: 98320
done: 98321
done: 98322
done: 98323
done: 98324
done: 98325
done: 98326
done: 98327
done: 98328
done: 98329
done: 98330
done: 98331
done: 98332
done: 98333
done: 98334
done: 98335
done: 98336
done: 98337
done: 98338
done: 98339
done: 98340
done: 98341
done: 98342
done: 98343
done: 98344
done: 98345
done: 98346
done: 98347
done: 98348
done: 98349
done: 98350
done: 98351
done: 98352
done: 98353
done: 98354
done

In [228]:
gdf_points_clean.head(5)
# only two columns are left => geometry and name of point

,geometry,desc_points
0,POINT (-9.41390 38.96183),feat_highway_crossing
1,POINT (-9.18832 38.74858),feat_highway_crossing
2,POINT (-9.45296 38.75312),feat_highway_bus_stop
3,POINT (-9.45655 38.75241),feat_highway_bus_stop
4,POINT (-9.48265 38.77859),feat_highway_bus_stop


### Re-transforming point's names into columns

To prepare the dataset of points for the merger with the general data set we need to re-transfer the unique feature names into columns. In total we have **96** feature columns.

In [229]:
# encoding all unique values
encoder = OneHotEncoder()
enc_df = encoder.fit_transform(gdf_points_clean[['desc_points']])

In [230]:
# reapplying column names
enc_gdf_points = gpd.GeoDataFrame(enc_df.toarray(), columns=encoder.categories_[0])
enc_gdf_points = enc_gdf_points.join(gdf_points_clean)

**Note**<br>
We need the ```desc_points``` column for later plotting.

In [231]:
enc_gdf_points.max()

feat_barrier_block                           1.0
feat_barrier_bollard                         1.0
feat_barrier_border_control                  1.0
feat_barrier_bump_gate                       1.0
feat_barrier_cattle_grid                     1.0
                                     ...        
feat_man_made_windmill                       1.0
feat_man_made_windpump                       1.0
feat_man_made_works                          1.0
feat_man_made_yes                            1.0
desc_points                    feat_man_made_yes
Length: 98, dtype: object

### Merging Points with Road segments

In order to merge the points with the provided road segments we need to buffer the LineStrings of the roads and turn them into little Polygons in order to overlap with the POIs around the road. Later we will use the ```.intersect``` method for spatial joins to keep only the points which are in the vicinity of the road segments.

In [232]:
# create a gdf with buffered road segments
geo_lis_buf = geo_lis.copy()
# allowing certain buffer to road segments to "catch" the points. buffer=.0005 seems to be visually adequate.
geo_lis_buf['geometry'] = geo_lis_buf.geometry.buffer(.0005)

<ipython-input-232-e52e1aef0238>:4: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_lis_buf['geometry'] = geo_lis_buf.geometry.buffer(.0005)


In [233]:
# joining both geo dfs
joint_gpd = gpd.sjoin(enc_gdf_points, geo_lis_buf, how="inner", op='intersects')

<ipython-input-233-b3a73a58ea89>:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joint_gpd = gpd.sjoin(enc_gdf_points, geo_lis_buf, how="inner", op='intersects')


In [234]:
print(f'We have {joint_gpd.shape[0]} intersecting points with our road segments.')

We have 34859 intersecting points with our road segments.


**Note**<br>
Now, we want to regroup the GDF back to our initial granularity, the road segments with unique link_IDs.

In [235]:
# building the aggregation dictionary for the .groupby method
columns = joint_gpd.columns
agg = {i:'sum' for i in columns if 'feat' in i}
agg['geometry'] = lambda x: list(x)
agg['desc_points'] = lambda x: list(x)
# agg
# all POIs should be summed, but the geometries of the containing points are to be listed in one cell.

In [236]:
# regrouping by linkid
grouped_gpd = joint_gpd.groupby('linkid').agg(agg)
# renaming the 'geometry' column so that the gdf won't be confused later
grouped_gpd = grouped_gpd.rename(columns={'geometry':'points'})

**Note**<br>
Only road segments which contained one or more points will be left in the gdf

In [237]:
geo_df_lis = geo_lis.merge(grouped_gpd, left_on='linkid', right_index=True)
geo_df_lis['point_count'] = geo_df_lis['points'].apply(lambda x: len(x))

In [238]:
# have a glance at the merged df
pd.set_option('display.max_columns', None)
geo_df_lis.head(3)

,linkid,Daily_Average_Traffic_Intensity,Average_Velocity_of_Vehicle_Traffic,Median_of_velocity_of_Vehicle_Traffic,FirstQuartil_of_velocity_of_Vehicle_Traffic,ThirdQuartil_of_velocity_of_Vehicle_Traffic,Func_Class,Speed_Cat,geometry,Max_speed,Speed_Diff_Mean,Speed_Diff_Median,func_1,func_2,func_3,speed_2,speed_3,speed_4,speed_5,speed_6,speed_7,feat_barrier_block,feat_barrier_bollard,feat_barrier_border_control,feat_barrier_bump_gate,feat_barrier_cattle_grid,feat_barrier_chain,feat_barrier_cycle_barrier,feat_barrier_debris,feat_barrier_door,feat_barrier_entrance,feat_barrier_fence,feat_barrier_gate,feat_barrier_height_restrictor,feat_barrier_jersey_barrier,feat_barrier_kerb,feat_barrier_kissing_gate,feat_barrier_lift_gate,feat_barrier_rope,feat_barrier_sally_port,feat_barrier_security_check,feat_barrier_stile,feat_barrier_swing_gate,feat_barrier_toll_booth,feat_barrier_toll_gantry,feat_barrier_turnstile,feat_barrier_wall,feat_barrier_yes,feat_highway_bus_stop,feat_highway_crossing,feat_highway_crossing;traffic_signals,feat_highway_elevator,feat_highway_footway,feat_highway_give_way,feat_highway_milestone,feat_highway_mini_roundabout,feat_highway_motorway_junction,feat_highway_path,feat_highway_platform,feat_highway_raceway,feat_highway_rest_area,feat_highway_services,feat_highway_speed_camera,feat_highway_steps,feat_highway_stop,feat_highway_street_lamp,feat_highway_toll_gantry,feat_highway_traffic_mirror,feat_highway_traffic_signals,feat_highway_traffic_signals;crossing,feat_highway_trailhead,feat_highway_turning_circle,feat_highway_turning_loop,feat_man_made_advertising,feat_man_made_antenna,feat_man_made_beacon,feat_man_made_bunker_silo,feat_man_made_cairn,feat_man_made_chimney,feat_man_made_communications_tower,feat_man_made_crane,feat_man_made_cross,feat_man_made_flagpole,feat_man_made_gasometer,feat_man_made_kiln,feat_man_made_lighthouse,feat_man_made_lookout_tower,feat_man_made_manhole,feat_man_made_mast,feat_man_made_monitoring_station,feat_man_made_pier,feat_man_made_pole,feat_man_made_reservoir,feat_man_made_reservoir_covered,feat_man_made_satellite_centre,feat_man_made_silo,feat_man_made_standpipe,feat_man_made_storage_tank,feat_man_made_street_cabinet,feat_man_made_surveillance,feat_man_made_survey_point,feat_man_made_tanks,feat_man_made_telescope,feat_man_made_threshing_floor,feat_man_made_tower,feat_man_made_utility_pole,feat_man_made_ventilation_shaft,feat_man_made_wastewater_plant,feat_man_made_water_tank,feat_man_made_water_tap,feat_man_made_water_tower,feat_man_made_water_well,feat_man_made_water_works,feat_man_made_watermill,feat_man_made_windmill,feat_man_made_windpump,feat_man_made_works,feat_man_made_yes,points,desc_points,point_count
0,80216819,1.058039,-0.448154,-0.476554,-0.650809,-0.335520,2,6,"MULTILINESTRING ((-9.16402 38.77030, -9.16389 ...",-0.716756,-0.549408,-0.506457,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[POINT (-9.1639781 38.7711136), POINT (-9.1642...","[feat_highway_traffic_signals, feat_highway_tr...",5
1,80216858,-0.405185,0.964251,1.051571,1.147324,0.946946,2,4,"MULTILINESTRING ((-9.16645 38.74274, -9.16638 ...",0.624480,-0.253319,-0.414450,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[POINT (-9.1657458 38.7430835), POINT (-9.1658...","[f

In [239]:
geo_df_lis = geo_df_lis.sort_values(by='linkid')

### Testplots for the merged data

In [240]:
# set of points to make them readable for the Marker (folium)
coords = [[(point.coords.xy[1][0], point.coords.xy[0][0]) for point in x] for x in geo_df_lis.head(100).points]
names = [x for x in geo_df_lis.head(100).desc_points]
map_lis_buf = geo_df_lis.copy()
map_lis_buf['geometry'] = map_lis_buf.geometry.buffer(.0005)

<ipython-input-240-4abb0788da36>:5: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map_lis_buf['geometry'] = map_lis_buf.geometry.buffer(.0005)


In [241]:
# init map
m = Map([38.74288, -9.16624])

# unpacking list of lists containing points, mapping them to their names
## POINTS (only a subset of 100)
marker_cluster = MarkerCluster(name='Points')
for i, coordi in enumerate(coords[:100]):
    popups = [Popup(f'<p><b>Name:</b></p> <p>{a}</p>', max_width=150) for a in names[i]]
    markers = [Marker(coord, popup=popups[c]).add_to(marker_cluster) for c, coord in enumerate(coordi)]

print('built points')

## BUFFERED ROADS (enable via layer control)
# only the first 100
roads_buf = Choropleth(geo_data=map_lis_buf.head(100).geometry,
                          data=None, 
                          name="roads_buf", 
                          show=False)


## ROADS
roads = Choropleth(geo_data=geo_df_lis.head(100).geometry,
                          data=None, 
                          name="roads", 
                          show=True)

print('built roads')

marker_cluster.add_to(m)
roads_buf.add_to(m)
roads.add_to(m)



LayerControl().add_to(m)
m

built points
built roads


**Note**<br>
The displayed points all seem to be within the boundaries of the buffered road segments 🛣.<br>
NOT ALL points are displayed. That would take up too much memory. 